In [23]:
import gwiko
import gwiko_generate

gwiko.reload_lib("gwiko")
gwiko.reload_lib("gwiko_generate")

g = gwiko_generate.Generate("C:/Users/Cosmos/Desktop/output/tmp/")
g.set_size(1920, 1080)

class Write():
    def __init__(self):
        self.tokens = []
    def add(self, el):
        self.tokens.append(el)
    def get_str(self):
        return "".join(self.tokens)

w = Write()
w.add("int int_frame = " +  str(0) + ";\n")
w.add("void main(){\n");
w.add("  ivec2 pixel_coords = ivec2(gl_GlobalInvocationID.xy);\n")
w.add("  vec4 color = vec4(pixel_coords.x / 1920.0, pixel_coords.y / 1080.0, 0.0, 1.0);\n")
w.add("  color.g = color.g + float(int_frame) * 0.01;\n")
w.add("  color.r *= 0.74;\n")
w.add("  imageStore(out0, pixel_coords, color);\n")
w.add("}\n")

print(w.get_str())

g.generate(w.get_str(), True)

gwiko reloaded successfully.
gwiko_generate reloaded successfully.
Folder 'C:\Users\Cosmos\Desktop\output\tmp' already exists.
Folder 'C:\Users\Cosmos\Desktop\output\tmp\output' already exists.
Folder 'C:\Users\Cosmos\Desktop\output\tmp\input' already exists.
int int_frame = 0;
void main(){
  ivec2 pixel_coords = ivec2(gl_GlobalInvocationID.xy);
  vec4 color = vec4(pixel_coords.x / 1920.0, pixel_coords.y / 1080.0, 0.0, 1.0);
  color.g = color.g + float(int_frame) * 0.01;
  color.r *= 0.74;
  imageStore(out0, pixel_coords, color);
}

JSON data saved to: C:\Users\Cosmos\Desktop\output\tmp\input/program.json
File C:\Users\Cosmos\Desktop\output\tmp\output\out_0.png deleted successfully.
File C:\Users\Cosmos\Desktop\output\tmp\output\generated_shader.glsl.png does not exist.
File C:\Users\Cosmos\Desktop\output\tmp\output\log.txt deleted successfully.
Image loaded successfully.


NameError: name 'output_folder_path' is not defined